# Task #2 Data Analysis

### Connect to database

In [ ]:
import duckdb
# Connect to the DuckDB database change the name below as given while running ingest.py
con = duckdb.connect("music-listens.duckdb")

## In the following, we ask you to run some SQL queries on the database you built in Task
### 1. The goal is to get more information out of the provided data.

a) To get started, answer the following questions:

### Who are the top 10 users with respect to number of songs listened to?

In [6]:
con.sql("""SELECT   user_name, 
                    COUNT(recording_msid) AS total_listens 
                    FROM listens
                    GROUP BY user_name
                    ORDER BY COUNT(recording_msid) DESC
                    LIMIT 10""")

┌────────────────┬───────────────┐
│   user_name    │ total_listens │
│    varchar     │     int64     │
├────────────────┼───────────────┤
│ hds            │         46862 │
│ Groschi        │         14959 │
│ Silent Singer  │         12675 │
│ 6d6f7274686f6e │         11042 │
│ phdnk          │         10671 │
│ reverbel       │          8389 │
│ Cl�psHydra     │          8318 │
│ InvincibleAsia │          7658 │
│ cimualte       │          7356 │
│ inhji          │          6349 │
├────────────────┴───────────────┤
│ 10 rows              2 columns │
└────────────────────────────────┘

### How many users did listen to some song on the 1st of March 2019?

In [7]:
con.sql("""
    SELECT count(distinct user_name) AS active_users
    FROM listens
    WHERE DATE(listened_at) = '2019-03-01';
""")

┌──────────────┐
│ active_users │
│    int64     │
├──────────────┤
│           72 │
└──────────────┘

### For every user, what was the first song the user listened to?

In [8]:
con.sql("""
    with first_song as (SELECT l.user_name, r.track_name,l.listened_at,row_number() OVER (PARTITION BY l.user_name ORDER BY l.listened_at asc) AS rn
    FROM listens l
    left join
    recordings r
    ON l.recording_msid = r.recording_msid)
    select user_name, track_name, listened_at from first_song
    where rn = 1
    ORDER BY user_name;
""")

┌─────────────────┬──────────────────────────────────────┬─────────────────────┐
│    user_name    │              track_name              │     listened_at     │
│     varchar     │               varchar                │      timestamp      │
├─────────────────┼──────────────────────────────────────┼─────────────────────┤
│ 6d6f7274686f6e  │ The Leper Affinity                   │ 2019-01-01 11:41:51 │
│ Adsky_traktor   │ Сердце с долгом разлучается          │ 2019-01-01 10:24:44 │
│ AllSparks       │ Fever                                │ 2019-01-02 09:48:19 │
│ AlwinHummels    │ Geef me je angst                     │ 2019-02-24 12:40:47 │
│ Arcor           │ Exsultate Justi                      │ 2019-01-01 02:22:23 │
│ AscendedGravity │ Amoeba                               │ 2019-01-02 00:01:17 │
│ Bezvezenator    │ Devour                               │ 2019-01-01 07:19:22 │
│ BiamBioum       │ Beirut (14.12.16 - Live in Paris)    │ 2019-01-07 14:56:07 │
│ BlackGauna      │ Visionz 

### b) Next, let's do a deep dive into user behaviour next. 

### For each user, we want to know the top 3 days on which they had the most listens, and how many listens they had oneach of these days. The result should 

### include the following:

### 3 rows per user
### 3 columns: (user, number_of_listens, date)

### Please sort the result by the user and the number_of_listens column

In [9]:
con.sql("""
   WITH daily_counts AS (
  SELECT user_name, DATE(listened_at) AS listen_date, COUNT(*) AS listens_per_day
  FROM listens
  GROUP BY user_name, DATE(listened_at)
),
ranked AS (
  SELECT user_name, listen_date, listens_per_day,
    ROW_NUMBER() OVER (
      PARTITION BY user_name
      ORDER BY listens_per_day DESC
    ) AS rn
  FROM daily_counts
)
SELECT
  user_name,
  listens_per_day AS number_of_listens,
  listen_date    AS date
FROM ranked
WHERE rn <= 3
ORDER BY user_name, number_of_listens DESC;

""")

┌─────────────────┬───────────────────┬────────────┐
│    user_name    │ number_of_listens │    date    │
│     varchar     │       int64       │    date    │
├─────────────────┼───────────────────┼────────────┤
│ 6d6f7274686f6e  │               200 │ 2019-01-27 │
│ 6d6f7274686f6e  │               194 │ 2019-01-14 │
│ 6d6f7274686f6e  │               193 │ 2019-01-16 │
│ Adsky_traktor   │               109 │ 2019-01-03 │
│ Adsky_traktor   │                99 │ 2019-01-05 │
│ Adsky_traktor   │                86 │ 2019-01-04 │
│ AllSparks       │               114 │ 2019-01-31 │
│ AllSparks       │                81 │ 2019-01-23 │
│ AllSparks       │                71 │ 2019-01-07 │
│ AlwinHummels    │                 1 │ 2019-02-24 │
│    ·            │                 · │     ·      │
│    ·            │                 · │     ·      │
│    ·            │                 · │     ·      │
│ yellams         │               157 │ 2019-02-15 │
│ zebedeemcdougal │                81 │ 2019-0

### c) Finally, we want to understand the development of active users within our userbase.

### For this, please write a query that calculates, on a daily basis, the absolute number of active users, and the percentage of active users among all users.

### We define a user to be active one some day X, if the user listened to at least one song in the time interval [X-6 days, X]. 

### The result should adhere to the following schema:
### 1 row per day
### 3 columns: (date, number_active_users, percentage_active_users)

### Please sort the result by date.

In [ ]:
con.sql("""
   WITH all_dates AS (
  SELECT DISTINCT DATE(listened_at) AS date
  FROM listens
),
total_users AS (
  SELECT COUNT(DISTINCT user_name) AS cnt
  FROM listens
)
SELECT
  d.date,
  COUNT(DISTINCT l.user_name) AS number_active_users,
  ROUND(
    CAST(COUNT(DISTINCT l.user_name) AS DOUBLE) / tu.cnt,
    4
  ) AS percentage_active_users
FROM all_dates AS d
CROSS JOIN total_users AS tu
LEFT JOIN listens AS l
  ON DATE(l.listened_at) BETWEEN d.date - 6 AND d.date
GROUP BY d.date, tu.cnt
ORDER BY d.date;
""")

┌────────────┬─────────────────────┬─────────────────────────┐
│    date    │ number_active_users │ percentage_active_users │
│    date    │        int64        │         double          │
├────────────┼─────────────────────┼─────────────────────────┤
│ 2019-01-01 │                  68 │                    0.34 │
│ 2019-01-02 │                  92 │                    0.46 │
│ 2019-01-03 │                 100 │                     0.5 │
│ 2019-01-04 │                 105 │                   0.525 │
│ 2019-01-05 │                 106 │                    0.53 │
│ 2019-01-06 │                 109 │                   0.545 │
│ 2019-01-07 │                 113 │                   0.565 │
│ 2019-01-08 │                 112 │                    0.56 │
│ 2019-01-09 │                 114 │                    0.57 │
│ 2019-01-10 │                 114 │                    0.57 │
│     ·      │                   · │                      ·  │
│     ·      │                   · │                   

In [ ]:
# Close the connection to the database
con.close()